In [13]:
import mysql.connector
import pandas as pd
from collections import defaultdict
from datetime import datetime

# -------------------------------
# 1. Define Eastern / Western teams (2023-24 alignment as an example)
# -------------------------------
eastern_teams = [
    "Atlanta Hawks", "Boston Celtics", "Brooklyn Nets", "Charlotte Hornets",
    "Chicago Bulls", "Cleveland Cavaliers", "Detroit Pistons", "Indiana Pacers",
    "Miami Heat", "Milwaukee Bucks", "New York Knicks", "Orlando Magic",
    "Philadelphia 76ers", "Toronto Raptors", "Washington Wizards"
]

western_teams = [
    "Dallas Mavericks", "Denver Nuggets", "Golden State Warriors", "Houston Rockets",
    "Los Angeles Clippers", "Los Angeles Lakers", "Memphis Grizzlies", "Minnesota Timberwolves",
    "New Orleans Pelicans", "Oklahoma City Thunder", "Phoenix Suns", "Portland Trail Blazers",
    "Sacramento Kings", "San Antonio Spurs", "Utah Jazz"
]

# -------------------------------
# 2. Connect to the MySQL database
# -------------------------------
connection = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)
cursor = connection.cursor(dictionary=True)

# -------------------------------
# 3. Query bets and legs together
#    For "Conference Winner" bets in the NBA, 
#    in bankroll = 'GreenAleph' and status = 'Active'.
# -------------------------------
query = """
SELECT 
    b.WagerID,
    b.PotentialPayout,
    l.LegID,
    l.EventType,
    l.EventLabel,
    l.ParticipantName
FROM bets AS b
JOIN legs AS l ON b.WagerID = l.WagerID
WHERE b.WhichBankroll = 'GreenAleph'
  AND b.WLCA = 'Active'
  AND l.LeagueName = 'NBA'
  AND l.EventType = 'Conference Winner'
  AND (l.EventLabel = 'Eastern Conference' OR l.EventLabel = 'Western Conference');
"""

cursor.execute(query)
results = cursor.fetchall()

cursor.close()
connection.close()

# ------------------------------------------------
# 4. Group each row by WagerID
#    This way, we can see how many legs each bet has
# ------------------------------------------------
temp_storage = defaultdict(list)
for row in results:
    wager_id = row["WagerID"]
    temp_storage[wager_id].append(row)

# ------------------------------------------------
# 5. Build:
#     - "straight_bets" dict: sum of potential payouts for single-leg bets 
#       (e.g. "ParticipantName" = "Boston Celtics", "EventLabel" = "Eastern Conference")
#     - "bets_dict" for multi-leg parlays
# ------------------------------------------------
straight_bets = defaultdict(float)  # key = team name, value = total potential payout for single-leg bets
bets_dict = {}  # key = WagerID, value = { "PotentialPayout": X, "legs": [...] }

for wager_id, row_list in temp_storage.items():
    # PotentialPayout is presumably the same for all legs in this bet
    potential_payout = float(row_list[0]["PotentialPayout"] or 0.0)
    
    # Collect the legs
    legs = []
    for r in row_list:
        legs.append({
            "EventLabel": r["EventLabel"],
            "ParticipantName": r["ParticipantName"]
        })
    
    # Decide if single-leg or multi-leg
    if len(legs) == 1:
        # Single-leg bet
        leg = legs[0]
        # If it says "Eastern Conference" or "Western Conference", add to straight_bets
        if leg["EventLabel"] in ("Eastern Conference", "Western Conference"):
            team_name = leg["ParticipantName"]
            straight_bets[team_name] += potential_payout
        # If there's some other label, do nothing or handle differently
    else:
        # Multi-leg bet => keep in bets_dict
        bets_dict[wager_id] = {
            "PotentialPayout": potential_payout,
            "legs": legs
        }

# ------------------------------------------------
# 6. Helper function to check if a multi-leg bet 
#    "wins" given (east_winner, west_winner)
# ------------------------------------------------
def bet_wins_for_scenario(bet_legs, east_winner, west_winner):
    """
    Return True if this parlay is satisfied by exactly 
    the east_winner and west_winner passed in.
    """
    for leg in bet_legs:
        event_label = leg["EventLabel"]
        participant = leg["ParticipantName"]
        if event_label == "Eastern Conference":
            if participant != east_winner:
                return False
        elif event_label == "Western Conference":
            if participant != west_winner:
                return False
        else:
            # If there's some other label, you'd handle that logic here.
            return False  # or handle accordingly
    return True

# ------------------------------------------------
# 7. Build a 2D grid where each cell = 
#     (straight bet payout for EastTeam) + 
#     (straight bet payout for WestTeam) + 
#     (payout of any multi-leg parlays that match EastTeam & WestTeam)
# ------------------------------------------------
grid_data = []
for e_team in eastern_teams:
    row_values = []
    for w_team in western_teams:
        # 1) sum single-leg bets for e_team + w_team
        total_payout = straight_bets.get(e_team, 0.0) + straight_bets.get(w_team, 0.0)
        
        # 2) add any multi-leg parlays that match the scenario
        for _, bet_info in bets_dict.items():
            if bet_wins_for_scenario(bet_info["legs"], e_team, w_team):
                total_payout += float(bet_info["PotentialPayout"] or 0.0)
        
        row_values.append(total_payout)
    grid_data.append(row_values)

# ------------------------------------------------
# 8. Convert to Pandas DataFrame for a nice 2D grid
# ------------------------------------------------
df = pd.DataFrame(grid_data, index=eastern_teams, columns=western_teams)

# Format numerical cells with commas and round to the nearest whole number
df = df.round(0).applymap(lambda x: f"{int(x):,}")
print("\nNBA Conference Winner EV Grid:\n")
print(df.to_string())

# Export to CSV with timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
df.to_csv(f"nba_east_west_scenarios_{timestamp}.csv")



NBA Conference Winner EV Grid:

                    Dallas Mavericks Denver Nuggets Golden State Warriors Houston Rockets Los Angeles Clippers Los Angeles Lakers Memphis Grizzlies Minnesota Timberwolves New Orleans Pelicans Oklahoma City Thunder Phoenix Suns Portland Trail Blazers Sacramento Kings San Antonio Spurs Utah Jazz
Atlanta Hawks                 16,320        965,070               175,175         414,942      16,320             23,385            26,185                 16,320               18,695                16,320       16,320                 16,320           16,320         1,107,400    16,320
Boston Celtics                     0          2,700               158,855         122,980           0              7,065             9,865                      0                2,375                     0            0                      0                0         1,758,952         0
Brooklyn Nets                 68,100        308,775               226,955         175,900      68,10

/var/folders/lj/2t8twtsx5v1dw1dkbr59j1xr0000gn/T/ipykernel_22299/508159648.py:158: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.round(0).applymap(lambda x: f"{int(x):,}")
